<a href="https://colab.research.google.com/github/julianavelasquezg/Business-Intelligence/blob/main/Reto_2_%C3%81rboles_de_decisi%C3%B3n_Juliana_Vel%C3%A1squez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reto 2 - Árboles de decisión**

El análisis de este ejercicio se basará en las siete variables que se explicarán posteriormente. Esto se realiza con el propósito de identificar la precisión del modelo de árboles de decisión para calcular la probabilidad de admisión de alumnos en una universidad.

**Descripción de las variables**

* **Edu_Parent1:** Indica el nivel educativo alcanzado por el primer padre o tutor del solicitante.
* **Edu_Parent2:** Indica el nivel educativo alcanzado por el segundo padre o tutor del solicitante.
* **Gender:** Representa el género del solicitante, donde 0 corresponde a masculino y 1 a femenino.
* **White:** Variable binaria que indica si el solicitante se identifica como blanco (1) o no (0).
* **Asian:** Variable binaria que indica si el solicitante se identifica como asiático (1) o no (0).
* **HSGPA:** Promedio de calificaciones obtenido por el solicitante en la educación secundaria (High School GPA).
* **SAT/ACT:** Puntaje obtenido por el solicitante en los exámenes de admisión estandarizados SAT o ACT.
* **Admitted_Pred:** Indica la admisión del solicitante a la universidad, donde 1 significa admitido y 0 significa no admitido.


0. Cargamos las librerías de datos

In [ ]:
# Librerías científicas
import numpy as np
import pandas as pd

#Librerías Arbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Conectar al Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Cargamos la base de datos

In [ ]:
nxl= '/content/drive/MyDrive/2. CollegeAdmisions_Int_M.xlsx'
XDB= pd.read_excel(nxl, sheet_name=0)
XDB=XDB.dropna()
XDB.head(100)

#Cambiar de la variable 'Gender' el M o F por 0 y 1
XDB['Gender'] = XDB['Gender'].map({'M': 0, 'F': 1})

XD=XDB[['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']]
YD=XDB[['Admitted']]


<ipython-input-3-ead6cba3b282>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XDB['Gender'] = XDB['Gender'].map({'M': 0, 'F': 1})


2. Contamos el número de personas admitidas por la universidad

In [ ]:
#Cambiar la variable 'Admitted' el Yes or No por 1 y 0
XDB['Admitted'] = XDB['Admitted'].map({'Yes': 1, 'No': 0})

#Contamos el número de datos de la base de datos
total_datos=len(XDB)
print(f"El número total de datos de la base de datos es: {total_datos}")

#Ahora contamos el número de personas admitidas
admitidos = XDB['Admitted'].sum()
print(f"El número de personas admitidas es: {admitidos}")

El número total de datos de la base de datos es: 1528
El número de personas admitidas es: 1303


3. Comenzamos con la implementación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini', max_depth=4)
mar.fit(XD,YD)

#Realizamos la gráfica del árbol de decisión
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz

ve=['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']
dot_data=export_graphviz(mar, feature_names=ve, filled=True, rounded=True)
graph=graph_from_dot_data(dot_data)
graph.write_png('Reto 2. Arboles de decision.png')

True

4. Determinamos el número de nodos puros para el árbol asociado con la estructura del modelo.

In [ ]:
nodos_puros = sum(mar.tree_.impurity == 0.0)
print(f"Número de nodos puros en el árbol: {nodos_puros}")

Número de nodos puros en el árbol: 7


5. Seguimos con el pronostico de admision de los estudiantes

In [ ]:
# Cargar la base de datos de pronóstico
XDB0 = pd.read_excel(nxl, sheet_name=1)
XDB0.head(100)

# Convertir 'Gender' a valores numéricos
XDB0['Gender'] = XDB0['Gender'].map({'M': 0, 'F': 1})

# Convertir 'Admitted' a valores numéricos
XDB0['Admitted'] = XDB0['Admitted'].map({'Yes': 1, 'No': 0})

# Seleccionar las variables de entrada y salida
XD0 = XDB0[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']].values
YD0 = XDB0[['Admitted']].values  # Esto es solo para referencia, no se usa en la predicción

# Realizar el pronóstico de admisión para todos los estudiantes
y_pred = mar.predict(XD0)

# Crear el informe en Excel con los resultados
df = pd.DataFrame(np.column_stack((XDB0.index, XD0, y_pred)))
df.columns = ['Solicitante', 'Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT', 'Admitted_Pred']

# Guardar el resultado en un archivo de Excel
df.to_excel("/content/drive/MyDrive/CollegeAdmisions_Pronostico.xlsx")

print("El pronóstico de admisión ha sido guardado en 'CollegeAdmisions_Pronostico.xlsx'.")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


El pronóstico de admisión ha sido guardado en 'CollegeAdmisions_Pronostico.xlsx'.


6. Evaluamos el rendimiento del modelo

In [ ]:
YDP=mar.predict(XD) #Pronóstico sobre la base de datos original
cm=confusion_matrix(YD,YDP)
print("La matriz de confusión es: \n", cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

#Métricas de desempeño
EX=(VP+VN)/(VP+VN+FP+FN) #Exactitud
print('La exactitud del modelo es:\n',EX)

TE=(FN+FP)/(VP+VN+FP+FN) #Tasa de error:
print('La tasa de error del modelo es:\n',TE)

SE=VP/(VP+FN) #Sensibilidad
print('La sensibilidad del modelo es:\n',SE)

SP=VN/(VN+FP) #Especificidad
print('La especificidad del modelo es:\n',SP)

PR=VP/(VP+FP) #Precisión
print('La precisión del modelo es:\n',PR)

PRNeng=VN/(VN+FN) #Precisión Negativa
print('La precisión negativa del modelo es:\n',PRNeng)

La matriz de confusión es: 
 [[  20  205]
 [   2 1301]]
La exactitud del modelo es:
 0.8645287958115183
La tasa de error del modelo es:
 0.1354712041884817
La sensibilidad del modelo es:
 0.9984650805832693
La especificidad del modelo es:
 0.08888888888888889
La precisión del modelo es:
 0.8638778220451527
La precisión negativa del modelo es:
 0.9090909090909091


**Análisis de las métricas**

El modelo de clasificación fue evaluado con un total de 1528 registros, de los cuales 1303 estudiantes fueron admitidos y 225 no admitidos.

A partir de los resultados obtenidos por la matriz de confusión podemos decir que el modelo clasificó correctamente a 1301 estudiantes admitidos (verdaderos positivos) y a 20 no admitidos (verdaderos negativos). Sin embargo, 205 estudiantes que en realidad no fueron admitidos fueron clasificados erróneamente como admitidos (falsos positivos), y solo 2 admitidos fueron clasificados erróneamente como no admitidos (falsos negativos).

La exactitud general del modelo es 86.45%, lo que indica un buen desempeño en la clasificación. Su sensibilidad del 99.85% muestra que identifica correctamente casi todos los admitidos, y su precisión del 86.39% indica que la mayoría de los clasificados como admitidos realmente lo son. Sin embargo, la especificidad es solo del 8.89%, lo que sugiere que el modelo tiene dificultades para detectar correctamente a los no admitidos, clasificando erróneamente a muchos de ellos. Esto se refleja en una tasa de error del 13.55%, que aunque no es un valor malo, indica que puede haber mejora en el modelo. La precisión negativa es del 90.91%, lo que significa que la mayoría de los clasificados como no admitidos lo fueron correctamente.

En conclusión, el modelo es confiable para predecir admisiones, pero necesita ajustes en la identificación de los no admitidos para mejorar su capacidad de distinguir entre ambos grupos.